In [1]:
# Hello world GAN model. 
# Generate mnist like images

In [2]:
import numpy as np
import pandas
import matplotlib.pyplot as plt

In [3]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.optimizers import Adam

Using TensorFlow backend.


In [4]:
batch_size = 128
epochs = 30000
latent_dim = 100
sample_interval = 600

In [5]:
# define input shapes 
image_width = 28
image_height = 28
image_channels = 1
image_shape = (image_width, image_height, image_channels)
print(image_shape)

(28, 28, 1)


In [6]:
# Build the discriminator 
model_disc = Sequential()
model_disc.add(Flatten(input_shape=image_shape))
model_disc.add(Dense(512))
model_disc.add(LeakyReLU(alpha=0.2))
model_disc.add(Dense(256))
model_disc.add(LeakyReLU(alpha=0.2))
model_disc.add(Dense(1,activation='sigmoid'))

model_disc.summary()

disc_in = Input(shape=image_shape)
valid = model_disc(disc_in)

discriminator = Model(disc_in, valid)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


In [7]:
#Build the generator
model_generator = Sequential()

model_generator.add(Dense(256, input_dim = latent_dim))
model_generator.add(LeakyReLU(alpha=0.2))
model_generator.add(BatchNormalization(momentum=0.8))
model_generator.add(Dense(512))
model_generator.add(LeakyReLU(alpha=0.2))
model_generator.add(BatchNormalization(momentum=0.8))
model_generator.add(Dense(1024))
model_generator.add(LeakyReLU(alpha=0.2))
model_generator.add(BatchNormalization(momentum=0.8))
model_generator.add(Dense(np.prod(image_shape),activation="tanh"))
model_generator.add(Reshape(image_shape))

model_generator.summary()

noise_in = Input(shape=(latent_dim,))
image_gen = model_generator(noise_in)

generator = Model(noise_in, image_gen)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
__________

In [8]:
# Readying the models 

discriminator.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5))

In [9]:
z = Input(shape=(latent_dim,))
image = generator(z)

In [10]:
discriminator.trainable = False # WHY ??

In [11]:
validity = discriminator(image)

In [ ]:
#generator.compile(loss="binary_crossentropy",optimizer=Adam(0.0002, 0.5))

In [12]:
# Combined model , stacked generator and discriminator 
# train generator to fool the discriminator 

combined = Model(z,validity)
combined.compile(loss="binary_crossentropy", optimizer=Adam(0.0002, 0.5))

In [13]:
def save_images(epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, latent_dim))
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("mnist_generated_images/%d.png" % epoch)
        plt.close()

In [ ]:
def show_images(epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, latent_dim))
        gen_imgs = generator.predict(noise)
        print(gen_imgs)
        
        

#         # Rescale images 0 - 1
#         gen_imgs = 0.5 * gen_imgs + 0.5

#         fig, axs = plt.subplots(r, c)
#         cnt = 0
#         for i in range(r):
#             plt.imshow(gen_imgs[r, :,:,0], cmap='gray')
#             #axs[i,j].axis('off')
#             cnt += 1
#         #fig.savefig("images/%d.png" % epoch)
#         plt.close()

In [18]:
# Processing the dataset (MNIST) 

(X_train,_),(_,_) = mnist.load_data()
print(X_train.shape)

(60000, 28, 28)


In [15]:
#Normalizing 
X_train = X_train / 127.5 -1.
X_train = np.expand_dims(X_train, axis=3)

In [16]:
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

In [17]:
for x in range(epochs):
    # Train discriminator 
    idx = np.random.randint(0,X_train.shape[0], batch_size)
    imgs = X_train[idx]
    #print(imgs.shape)
    
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    #print(noise.shape)
    gen_images = generator.predict(noise)
    #print(gen_images.shape)
    
    disc_loss_real = discriminator.train_on_batch(imgs, valid)
    #print(disc_loss_real)
    disc_loss_fake = discriminator.train_on_batch(gen_images, fake)
    #print(disc_loss_fake)
    disc_loss = 0.5 * np.add(disc_loss_real, disc_loss_fake)

    # TODO : understand why above things are done. 
    
    # Train the discriminator 
    generator_loss = combined.train_on_batch(noise, valid)
    
    #print("=================LOSSES===============",x+1)
    #print(disc_loss)
    #print(generator_loss)
    
    # Progress =======>
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (x, disc_loss, 100*disc_loss, generator_loss))
    
    if x % sample_interval == 0:
        save_images(x)

d:\program files\python\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.677804, acc.: 67.78%] [G loss: 0.651470]
1 [D loss: 0.434903, acc.: 43.49%] [G loss: 0.729648]
2 [D loss: 0.348374, acc.: 34.84%] [G loss: 0.870393]
3 [D loss: 0.302313, acc.: 30.23%] [G loss: 0.993418]
4 [D loss: 0.271592, acc.: 27.16%] [G loss: 1.124659]
5 [D loss: 0.226268, acc.: 22.63%] [G loss: 1.260476]
6 [D loss: 0.195147, acc.: 19.51%] [G loss: 1.381099]
7 [D loss: 0.169641, acc.: 16.96%] [G loss: 1.482551]
8 [D loss: 0.149777, acc.: 14.98%] [G loss: 1.608363]
9 [D loss: 0.140520, acc.: 14.05%] [G loss: 1.672654]
10 [D loss: 0.132621, acc.: 13.26%] [G loss: 1.750368]
11 [D loss: 0.109129, acc.: 10.91%] [G loss: 1.875779]
12 [D loss: 0.106299, acc.: 10.63%] [G loss: 1.933191]
13 [D loss: 0.101221, acc.: 10.12%] [G loss: 1.996972]
14 [D loss: 0.092149, acc.: 9.21%] [G loss: 2.077167]
15 [D loss: 0.084619, acc.: 8.46%] [G loss: 2.143365]
16 [D loss: 0.082983, acc.: 8.30%] [G loss: 2.208735]
17 [D loss: 0.071230, acc.: 7.12%] [G loss: 2.293643]
18 [D loss: 0.071984, ac

154 [D loss: 0.088528, acc.: 8.85%] [G loss: 3.643227]
155 [D loss: 0.160691, acc.: 16.07%] [G loss: 3.106824]
156 [D loss: 0.164443, acc.: 16.44%] [G loss: 3.341713]
157 [D loss: 0.389639, acc.: 38.96%] [G loss: 2.965255]
158 [D loss: 0.097008, acc.: 9.70%] [G loss: 3.428193]
159 [D loss: 0.102498, acc.: 10.25%] [G loss: 3.291812]
160 [D loss: 0.190994, acc.: 19.10%] [G loss: 3.211115]
161 [D loss: 0.270157, acc.: 27.02%] [G loss: 3.185066]
162 [D loss: 0.257953, acc.: 25.80%] [G loss: 2.980984]
163 [D loss: 0.095889, acc.: 9.59%] [G loss: 3.379779]
164 [D loss: 0.262944, acc.: 26.29%] [G loss: 3.174683]
165 [D loss: 0.263815, acc.: 26.38%] [G loss: 3.325225]
166 [D loss: 0.714856, acc.: 71.49%] [G loss: 3.072362]
167 [D loss: 0.085099, acc.: 8.51%] [G loss: 3.308481]
168 [D loss: 0.096175, acc.: 9.62%] [G loss: 3.120639]
169 [D loss: 0.126044, acc.: 12.60%] [G loss: 3.267026]
170 [D loss: 0.207831, acc.: 20.78%] [G loss: 3.471915]
171 [D loss: 0.327193, acc.: 32.72%] [G loss: 3.15163

306 [D loss: 0.321461, acc.: 32.15%] [G loss: 2.974446]
307 [D loss: 0.763683, acc.: 76.37%] [G loss: 2.336885]
308 [D loss: 0.295627, acc.: 29.56%] [G loss: 2.646072]
309 [D loss: 0.515818, acc.: 51.58%] [G loss: 2.020390]
310 [D loss: 0.232856, acc.: 23.29%] [G loss: 2.751108]
311 [D loss: 0.520498, acc.: 52.05%] [G loss: 2.090632]
312 [D loss: 0.287327, acc.: 28.73%] [G loss: 2.740337]
313 [D loss: 0.536418, acc.: 53.64%] [G loss: 2.048800]
314 [D loss: 0.254033, acc.: 25.40%] [G loss: 2.726751]
315 [D loss: 0.352212, acc.: 35.22%] [G loss: 2.187148]
316 [D loss: 0.369433, acc.: 36.94%] [G loss: 2.561404]
317 [D loss: 0.372501, acc.: 37.25%] [G loss: 2.141843]
318 [D loss: 0.327976, acc.: 32.80%] [G loss: 2.706054]
319 [D loss: 0.472358, acc.: 47.24%] [G loss: 2.214811]
320 [D loss: 0.305100, acc.: 30.51%] [G loss: 2.479469]
321 [D loss: 0.472783, acc.: 47.28%] [G loss: 2.084380]
322 [D loss: 0.301318, acc.: 30.13%] [G loss: 2.932496]
323 [D loss: 0.695117, acc.: 69.51%] [G loss: 2.

457 [D loss: 0.703660, acc.: 70.37%] [G loss: 0.692660]
458 [D loss: 0.687216, acc.: 68.72%] [G loss: 0.709786]
459 [D loss: 0.709261, acc.: 70.93%] [G loss: 0.685185]
460 [D loss: 0.693421, acc.: 69.34%] [G loss: 0.690616]
461 [D loss: 0.674736, acc.: 67.47%] [G loss: 0.698967]
462 [D loss: 0.691527, acc.: 69.15%] [G loss: 0.703904]
463 [D loss: 0.695574, acc.: 69.56%] [G loss: 0.698657]
464 [D loss: 0.694455, acc.: 69.45%] [G loss: 0.697527]
465 [D loss: 0.692586, acc.: 69.26%] [G loss: 0.694265]
466 [D loss: 0.697068, acc.: 69.71%] [G loss: 0.685101]
467 [D loss: 0.687590, acc.: 68.76%] [G loss: 0.687742]
468 [D loss: 0.697030, acc.: 69.70%] [G loss: 0.685642]
469 [D loss: 0.687148, acc.: 68.71%] [G loss: 0.693168]
470 [D loss: 0.692964, acc.: 69.30%] [G loss: 0.682685]
471 [D loss: 0.687362, acc.: 68.74%] [G loss: 0.690468]
472 [D loss: 0.693653, acc.: 69.37%] [G loss: 0.687542]
473 [D loss: 0.697024, acc.: 69.70%] [G loss: 0.674893]
474 [D loss: 0.691851, acc.: 69.19%] [G loss: 0.

...

29400 [D loss: 0.637182, acc.: 63.72%] [G loss: 0.886887]
29401 [D loss: 0.662292, acc.: 66.23%] [G loss: 0.938390]
29402 [D loss: 0.634018, acc.: 63.40%] [G loss: 0.910089]
29403 [D loss: 0.669458, acc.: 66.95%] [G loss: 0.913633]
29404 [D loss: 0.647060, acc.: 64.71%] [G loss: 0.896105]
29405 [D loss: 0.671396, acc.: 67.14%] [G loss: 0.909212]
29406 [D loss: 0.697022, acc.: 69.70%] [G loss: 0.923003]
29407 [D loss: 0.634957, acc.: 63.50%] [G loss: 0.945350]
29408 [D loss: 0.652944, acc.: 65.29%] [G loss: 0.919079]
29409 [D loss: 0.650872, acc.: 65.09%] [G loss: 0.898665]
29410 [D loss: 0.655602, acc.: 65.56%] [G loss: 0.921552]
29411 [D loss: 0.638020, acc.: 63.80%] [G loss: 0.905667]
29412 [D loss: 0.669565, acc.: 66.96%] [G loss: 0.903993]
29413 [D loss: 0.639605, acc.: 63.96%] [G loss: 0.987907]
29414 [D loss: 0.681320, acc.: 68.13%] [G loss: 0.903030]
29415 [D loss: 0.654322, acc.: 65.43%] [G loss: 0.942850]
29416 [D loss: 0.672858, acc.: 67.29%] [G loss: 0.863960]
29417 [D loss:

29545 [D loss: 0.655864, acc.: 65.59%] [G loss: 0.934274]
29546 [D loss: 0.656249, acc.: 65.62%] [G loss: 0.912131]
29547 [D loss: 0.678258, acc.: 67.83%] [G loss: 0.898070]
29548 [D loss: 0.655628, acc.: 65.56%] [G loss: 0.931836]
29549 [D loss: 0.647222, acc.: 64.72%] [G loss: 0.944594]
29550 [D loss: 0.688249, acc.: 68.82%] [G loss: 0.915554]
29551 [D loss: 0.644255, acc.: 64.43%] [G loss: 0.948055]
29552 [D loss: 0.655352, acc.: 65.54%] [G loss: 0.919739]
29553 [D loss: 0.668398, acc.: 66.84%] [G loss: 0.934080]
29554 [D loss: 0.677560, acc.: 67.76%] [G loss: 0.878791]
29555 [D loss: 0.655685, acc.: 65.57%] [G loss: 0.891003]
29556 [D loss: 0.623849, acc.: 62.38%] [G loss: 0.959209]
29557 [D loss: 0.649697, acc.: 64.97%] [G loss: 0.968095]
29558 [D loss: 0.666572, acc.: 66.66%] [G loss: 0.920404]
29559 [D loss: 0.653287, acc.: 65.33%] [G loss: 0.980709]
29560 [D loss: 0.654912, acc.: 65.49%] [G loss: 0.897776]
29561 [D loss: 0.636873, acc.: 63.69%] [G loss: 0.913283]
29562 [D loss:

29688 [D loss: 0.646410, acc.: 64.64%] [G loss: 0.928899]
29689 [D loss: 0.656077, acc.: 65.61%] [G loss: 0.913015]
29690 [D loss: 0.679347, acc.: 67.93%] [G loss: 0.893795]
29691 [D loss: 0.648727, acc.: 64.87%] [G loss: 0.907942]
29692 [D loss: 0.690021, acc.: 69.00%] [G loss: 0.854223]
29693 [D loss: 0.648464, acc.: 64.85%] [G loss: 0.886579]
29694 [D loss: 0.635949, acc.: 63.59%] [G loss: 0.926859]
29695 [D loss: 0.696391, acc.: 69.64%] [G loss: 0.929088]
29696 [D loss: 0.682438, acc.: 68.24%] [G loss: 0.933843]
29697 [D loss: 0.645320, acc.: 64.53%] [G loss: 0.908917]
29698 [D loss: 0.687664, acc.: 68.77%] [G loss: 0.946691]
29699 [D loss: 0.665191, acc.: 66.52%] [G loss: 0.913984]
29700 [D loss: 0.650749, acc.: 65.07%] [G loss: 0.891187]
29701 [D loss: 0.674692, acc.: 67.47%] [G loss: 0.889894]
29702 [D loss: 0.651332, acc.: 65.13%] [G loss: 0.955939]
29703 [D loss: 0.651892, acc.: 65.19%] [G loss: 0.967770]
29704 [D loss: 0.671711, acc.: 67.17%] [G loss: 0.913174]
29705 [D loss:

29832 [D loss: 0.699365, acc.: 69.94%] [G loss: 0.882960]
29833 [D loss: 0.646792, acc.: 64.68%] [G loss: 0.921120]
29834 [D loss: 0.648333, acc.: 64.83%] [G loss: 0.946832]
29835 [D loss: 0.630410, acc.: 63.04%] [G loss: 0.951263]
29836 [D loss: 0.639863, acc.: 63.99%] [G loss: 0.958025]
29837 [D loss: 0.688506, acc.: 68.85%] [G loss: 0.910770]
29838 [D loss: 0.668483, acc.: 66.85%] [G loss: 0.892816]
29839 [D loss: 0.658321, acc.: 65.83%] [G loss: 0.930334]
29840 [D loss: 0.659701, acc.: 65.97%] [G loss: 0.940974]
29841 [D loss: 0.692817, acc.: 69.28%] [G loss: 0.862516]
29842 [D loss: 0.670391, acc.: 67.04%] [G loss: 0.885211]
29843 [D loss: 0.650799, acc.: 65.08%] [G loss: 0.911171]
29844 [D loss: 0.668506, acc.: 66.85%] [G loss: 0.902437]
29845 [D loss: 0.647613, acc.: 64.76%] [G loss: 0.926627]
29846 [D loss: 0.652063, acc.: 65.21%] [G loss: 0.937482]
29847 [D loss: 0.682356, acc.: 68.24%] [G loss: 0.951953]
29848 [D loss: 0.683948, acc.: 68.39%] [G loss: 0.942726]
29849 [D loss:

29977 [D loss: 0.659200, acc.: 65.92%] [G loss: 0.921767]
29978 [D loss: 0.650330, acc.: 65.03%] [G loss: 0.878568]
29979 [D loss: 0.672849, acc.: 67.28%] [G loss: 0.935269]
29980 [D loss: 0.632113, acc.: 63.21%] [G loss: 0.910506]
29981 [D loss: 0.674603, acc.: 67.46%] [G loss: 0.909149]
29982 [D loss: 0.644689, acc.: 64.47%] [G loss: 0.928018]
29983 [D loss: 0.675889, acc.: 67.59%] [G loss: 0.923544]
29984 [D loss: 0.674638, acc.: 67.46%] [G loss: 0.914775]
29985 [D loss: 0.671747, acc.: 67.17%] [G loss: 0.908244]
29986 [D loss: 0.672693, acc.: 67.27%] [G loss: 0.893960]
29987 [D loss: 0.677873, acc.: 67.79%] [G loss: 0.902629]
29988 [D loss: 0.635411, acc.: 63.54%] [G loss: 0.957551]
29989 [D loss: 0.670383, acc.: 67.04%] [G loss: 0.893288]
29990 [D loss: 0.688242, acc.: 68.82%] [G loss: 0.878339]
29991 [D loss: 0.636313, acc.: 63.63%] [G loss: 0.907095]
29992 [D loss: 0.665983, acc.: 66.60%] [G loss: 0.868205]
29993 [D loss: 0.654715, acc.: 65.47%] [G loss: 0.934199]
29994 [D loss: